In [97]:
import sys

In [98]:
sys.path.append(os.path.join(os.path.abspath('../..'), 'src'))

In [28]:
base = "http://www.ncleg.net/gascripts/lastaction/todaysfiledaction.pl?"

In [73]:
# Form March 21 2017
query_base = "Biennium={0}&ActionChamber={1}&DateReport={2}"
query_base.format("2017", "H", "03%2F21%2F2017")

'Biennium=2017&ActionChamber=H&DateReport=03%2F21%2F2017'

In [24]:
import requests
from bs4 import BeautifulSoup as bs
from bs4 import SoupStrainer

In [107]:
def build_query_url_from_date(yr, mo, da, chamber):
    
    url_base = "http://www.ncleg.net/gascripts/lastaction/todaysfiledaction.pl?"
    query_base = "Biennium={0}&ActionChamber={1}&DateReport={2}"
    
    def mdstr(md):
        mdstr = str(md)
        if md < 10:
            mdstr = '0' + mdstr
        return(mdstr)
    
    def ds_from_comps(yr, mo, da):
        ds = mo + "%2F" + da + "%2F" + yr
        return(ds)
    
    yr = str(yr)
    mo = mdstr(mo)
    da = mdstr(da)
    ds = ds_from_comps(yr, mo, da)
    
    query = query_base.format(yr, chamber, ds)
    return(url_base + query)

In [77]:
query_url_from_date(2017, 3, 15, 'H')

'http://www.ncleg.net/gascripts/lastaction/todaysfiledaction.pl?Biennium=2017&ActionChamber=H&DateReport=03%2F15%2F2017'

In [29]:
base+query

'http://www.ncleg.net/gascripts/lastaction/todaysfiledaction.pl?Biennium=2017&ActionChamber=H&DateReport=03%2F21%2F2017'

In [27]:
"http://www.ncleg.net/gascripts/lastaction/todaysfiledaction.pl?Biennium=2017&ActionChamber=H&DateReport=03%2F21%2F2017"

'http://www.ncleg.net/gascripts/lastaction/todaysfiledaction.pl?Biennium=2017&ActionChamber=H&DateReport=03%2F21%2F2017'

In [30]:
html = requests.get(base+query).content

In [31]:
soup = bs(html, 'html.parser')

In [11]:
tables = soup.find_all('table')

In [12]:
len(tables)

10

## Method 01: Grab the table, rely on table attributes

In [58]:
content_table_filter = SoupStrainer("table",
                                    {'border':0, 'cellpadding':2,})
table = soup.find(content_table_filter)
rows = table.find_all('tr', {'class':""})
header = rows[0]
rows = rows[1:]

In [59]:
[tc.text for tc in header.find_all('th')]

['Bill', 'Short Title', 'Action', 'Date', 'Action\xa0Text']

In [60]:
len(rows)

20

In [78]:
rows[0]

<tr class="">
<td class="tableText" style="vertical-align: top;"><a href="/gascripts/BillLookUp/BillLookUp.pl?Session=2017&amp;BillID=H414">H414</a></td>
<td class="tableText" style="vertical-align: top;"><a href="/gascripts/BillLookUp/BillLookUp.pl?Session=2017&amp;BillID=H414">Swain County/Official Trout Fishing Location.</a></td>
<td class="tableText" style="vertical-align: top; text-align: left;">H</td>
<td class="tableText" style="vertical-align: top;">03/21/2017</td>
<td class="tableText" style="vertical-align: top;">Filed</td>
</tr>

In [86]:
def extract_datarows_m01(soup):
    """
    "Dead-Reckon" table and extract rows accordingly
    """
    content_table_filter = SoupStrainer("table",
                                    {'border':0, 'cellpadding':2,})
    table = soup.find(content_table_filter)
    rows = table.find_all('tr', {'class':""})
    header = rows[0]
    rows = rows[1:]
    
    return(header, rows)

In [94]:
extract_datarows_m01(soup)[1][0].find_all('td')

[<td class="tableText" style="vertical-align: top;"><a href="/gascripts/BillLookUp/BillLookUp.pl?Session=2017&amp;BillID=H414">H414</a></td>,
 <td class="tableText" style="vertical-align: top;"><a href="/gascripts/BillLookUp/BillLookUp.pl?Session=2017&amp;BillID=H414">Swain County/Official Trout Fishing Location.</a></td>,
 <td class="tableText" style="vertical-align: top; text-align: left;">H</td>,
 <td class="tableText" style="vertical-align: top;">03/21/2017</td>,
 <td class="tableText" style="vertical-align: top;">Filed</td>]

## Method 02: Grab the Rows by relying on the individual row formats

Misses the header

In [62]:
rrr = soup.find_all("td", class_="tableText")

In [63]:
len(rrr)

100

In [64]:
new_entries = []
for i in range(0,len(rrr), 5):
    new_entries.append(rrr[i:(i+5)])

In [85]:
new_entries[0]

[<td class="tableText" style="vertical-align: top;"><a href="/gascripts/BillLookUp/BillLookUp.pl?Session=2017&amp;BillID=H414">H414</a></td>,
 <td class="tableText" style="vertical-align: top;"><a href="/gascripts/BillLookUp/BillLookUp.pl?Session=2017&amp;BillID=H414">Swain County/Official Trout Fishing Location.</a></td>,
 <td class="tableText" style="vertical-align: top; text-align: left;">H</td>,
 <td class="tableText" style="vertical-align: top;">03/21/2017</td>,
 <td class="tableText" style="vertical-align: top;">Filed</td>]

In [95]:
def extract_datarows_m02(soup):
    """
    Pull all of the target columns and rebuild into sets of rows;
    Convert easch back to a single bs4 tag object
    """
    column_filter = SoupStrainer("td", {"class":"tableText"})
    columns = soup.find_all(column_filter)
    
    rows = []
    for i in range(0,len(columns), 5):
        rows.append(bs("<tr class="">\n" + \
                       '\n'.join([str(c) for c in columns[i:(i+5)]]) +\
                       "\n<\tr>",
                      'html.parser'))
        
    return(rows)

In [96]:
extract_datarows_m02(soup)[0].find_all('td')

[<td class="tableText" style="vertical-align: top;"><a href="/gascripts/BillLookUp/BillLookUp.pl?Session=2017&amp;BillID=H414">H414</a></td>,
 <td class="tableText" style="vertical-align: top;"><a href="/gascripts/BillLookUp/BillLookUp.pl?Session=2017&amp;BillID=H414">Swain County/Official Trout Fishing Location.</a></td>,
 <td class="tableText" style="vertical-align: top; text-align: left;">H</td>,
 <td class="tableText" style="vertical-align: top;">03/21/2017</td>,
 <td class="tableText" style="vertical-align: top;">Filed</td>]

## Process Data From Rows

In [66]:
import pandas

In [71]:
# Columns are known:
cols = ['Bill', 'Short Title', 'Action', 'Date', 'Action Text']

# Cols with links go to the same place...?
link_cols = set(['Bill', 'Short Title'])
data = []

for row in rows:
    d = {}
    for col,entry in zip(cols, row.find_all('td')):
        d[col] = entry.text
       # if col in link_cols:
        if col == 'Bill':
            link = entry.find('a')['href']
            d['Link'] = link
    data.append(d)
    
data = pandas.DataFrame(data)

In [72]:
data.head()

,Action,Action Text,Bill,Date,Link,Short Title
0,H,Filed,H414,03/21/2017,/gascripts/BillLookUp/BillLookUp.pl?Session=20...,Swain County/Official Trout Fishing Location.
1,H,Filed,H415,03/21/2017,/gascripts/BillLookUp/BillLookUp.pl?Session=20...,Hatteras Village Community Center District.
2,H,Filed,H416,03/21/2017,/gascripts/BillLookUp/BillLookUp.pl?Session=20...,Robeson County/Local Business Preference.
3,H,Filed,H417,03/21/2017,/gascripts/BillLookUp/BillLookUp.pl?Session=20...,Actually Get Rid of Common Core Act.
4,H,Filed,H418,03/21/2017,/gascripts/BillLookUp/BillLookUp.pl?Session=20...,SOS/Save Our Street Signs.


# Test Full

In [103]:
from importlib import reload

In [104]:
import process_dailyupdates_pages

In [150]:
reload(process_dailyupdates_pages)

<module 'process_dailyupdates_pages' from '/home/immersinn/gits/ncga/src/process_dailyupdates_pages.py'>

In [151]:
hdtoday = process_dailyupdates_pages.get_DailyUpdateForDateChamber(2017, 1, 15, 'H')

In [152]:
hdtoday.head()

,Bill,Chamber,Date,Title,Link,Action,Session


In [128]:
import mysql_utils

In [144]:
reload(mysql_utils)

<module 'mysql_utils' from '/home/immersinn/gits/ncga/src/mysql_utils.py'>

In [153]:
mysql_utils.dump_filed_bills(hdtoday[hdtoday.Action=='Filed'])

'Pass'

In [146]:
hdtoday.shape

(27, 7)